In [9]:
# https://www.geeksforgeeks.org/bubble-sort/

import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import os
import sys
import math
import pickle

## Bubblesort Algorithm

In [10]:
def bubble_sort(arr): # x^2
  n = len(arr)
  for i in range(n):
    swapped = False
    for j in range(0, n-i-1):
      if arr[j] > arr[j+1]:
        arr[j], arr[j+1] = arr[j+1], arr[j]
        swapped = True
    if (swapped == False):
      break

## Frequency Count Method

In [11]:
X_y = []
i=50
for n in range(2,i+1): # started with 2 itens until 50 itens in the list
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(bubble_sort)

  # create a list with random number between 1 to 10000
  # with n number in the list
  input = random.choices(range(10000), k=n)
  lp_wrapper(input)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [12]:
resultados_com_menor_loss = []
repeat = 5
registros = []

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      binary_operators=["*", "+"],
      unary_operators=["log", "square", "cube"],
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)

    # second combination
    reg2 = PySRRegressor(
      binary_operators=["*"],
      unary_operators=["log", "square", "cube"],
    )

    fit2 = reg2.fit(X_reshaped, y)
    best_program2 = fit2.get_best()

    registro2 = []
    for index, value in enumerate(best_program2):
      registro2.append(value)
    registros.append(registro2)

    # third combinarion
    reg3 = PySRRegressor(
      binary_operators=["+"],
      unary_operators=["log", "square", "cube"],
    )

    fit3 = reg3.fit(X_reshaped, y)
    best_program3 = fit3.get_best()

    registro3 = []
    for index, value in enumerate(best_program3):
      registro3.append(value)
    registros.append(registro3)
    
sys.stdout = original_stdout

[ Info: Started!
0.0%┣                                              ┫ 0/600 [00:00<00:-3, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           2.996e+06  1.594e+01  y = x₀                                        2           3.684e+05  2.096e+00  y = square(x₀)                                4           3.345e+05  4.817e-02  y = (square(x₀) + x₀)                         5           1.524e+05  7.861e-01  y = square(log(x₀) + x₀)                      6           3.161e+04  1.573e+00  y = square(log(square(x₀)) + x₀)              7           1.826e+04  

In [13]:
registros_ = registros

In [14]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

## Save result

In [15]:
def salvar_dados(dados, arquivo):
  if os.path.exists(arquivo):
    with open(arquivo, 'rb') as f:
      dados_exist = pickle.load(f)

    dados_exist.update(dados)
  else:
    dados_exist = dados
  
  with open(arquivo, 'wb') as f:
    pickle.dump(dados_exist, f)
  
novos_dados = {'bubblesort_N^2': lista_melhor_valor[4]}
caminho_arquivo = 'dados.pickle'

salvar_dados(novos_dados, caminho_arquivo)

In [16]:
def carregar_dados(arquivo):
    # Carrega os dados do arquivo pickle
    with open(arquivo, 'rb') as f:
        dados = pickle.load(f)
    return dados

caminho_arquivo = 'dados.pickle'
dados_carregados = carregar_dados(caminho_arquivo)

print("Conteúdo do arquivo pickle:")
for k, v in dados_carregados.items(): print(k, ' = ',v)

Conteúdo do arquivo pickle:
{'shellsort_NlogN': 4.862569*x0*log(x0), 'mergesort_NlogN': 5.5040717*x0 + 8.03698770358715, 'bubblesort_x^2': 1.517833856016*x0**2}
